# Topic modelling

## 0.0 Imports

In [128]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dglover\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [129]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

## 1.0 Run

In [130]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [131]:
# Import Dataset
df = pd.read_csv('..\\data\\summaries_full.csv', index_col=0)
#print(df.target_names.unique())
df.head()

,Summary
PDF URL,
http://arxiv.org/pdf/1608.04434v1,Hadoop is one of the platform s that can proce...
http://arxiv.org/pdf/2202.07138v2,Integrating AI Planning with Natural Language ...
http://arxiv.org/pdf/1906.11608v2,The tools are machine learning based using nat...
http://arxiv.org/pdf/2006.16212v1,The name Tangkhul also known as Hao or Ihao re...
http://arxiv.org/pdf/1511.07916v1,This is a lecture note for the course DS GA at...


In [132]:
df.shape

(3557, 1)

In [133]:
df.dropna(inplace=True)

In [134]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = list(df['Summary'])
vectorizer = CountVectorizer(decode_error='ignore')
X = vectorizer.fit_transform(corpus)

In [135]:
vectorizer.get_feature_names_out()

array(['aa', 'aaaa', 'aaai', ..., 'zwnj', 'zynq', 'zzy'], dtype=object)

In [136]:
CV_df = pd.DataFrame(X.toarray())
CV_df.columns = vectorizer.get_feature_names_out()
#CV_df.head()

In [137]:
CV_df.shape

(3556, 24780)

In [138]:
CV_stopwords = []
for col in CV_df.columns:
    if sum(CV_df[col]) > 0.25 * (CV_df.shape[0]):
        CV_stopwords.append(col)

In [139]:
CV_stopwords

['also',
 'an',
 'and',
 'are',
 'as',
 'at',
 'based',
 'be',
 'by',
 'can',
 'data',
 'for',
 'from',
 'has',
 'have',
 'in',
 'information',
 'is',
 'it',
 'language',
 'languages',
 'learning',
 'model',
 'models',
 'natural',
 'new',
 'nlp',
 'of',
 'on',
 'or',
 'our',
 'paper',
 'processing',
 'published',
 'study',
 'such',
 'task',
 'tasks',
 'text',
 'that',
 'the',
 'this',
 'to',
 'university',
 'use',
 'used',
 'using',
 'we',
 'with']

In [140]:
extra_stopwords = ['author','article','use', 'show', 'however', 'approach', 'well', 'provide',' present', 'include', 'word', 'nlp', 'natural', 'language', 'processing']
full_stopwords = CV_stopwords + extra_stopwords
stop_words.extend(full_stopwords)

In [141]:
df.reset_index(inplace = True)

In [142]:
# Import Dataset
df2 = pd.read_csv('..\\data\\arxiv_papers_full_v2.csv', index_col=0)

In [143]:
df2.head(2)

,Title,PDF URL,Author,DOI,Published Date,Summary,Journal Ref,Primary Category,Category,Entry ID
0,Natural Language Processing using Hadoop and K...,http://arxiv.org/pdf/1608.04434v1,"[arxiv.Result.Author('Emre Erturk'), arxiv.Res...",NaN,2016-08-15 23:09:21+00:00,"Natural language processing, as a data analyti...",NaN,cs.CL,['cs.CL'],http://arxiv.org/abs/1608.04434v1
1,Integrating AI Planning with Natural Language ...,http://arxiv.org/pdf/2202.07138v2,"[arxiv.Result.Author('Kebing Jin'), arxiv.Resu...",NaN,2022-02-15 02:19:09+00:00,Natural language processing (NLP) aims at inve...,NaN,cs.AI,"['cs.AI', 'cs.CL']",http://arxiv.org/abs/2202.07138v2


In [144]:
df2.rename(columns = {'Summary' : 'Abstract'}, inplace=True)

In [145]:
#df2['Filename'] = df2['PDF URL'].map(lambda x: x.split('/')[-1] + '.pdf')

In [146]:
df = df.merge(df2, how = 'left', on = 'PDF URL', indicator = True)

In [147]:
df['_merge'].value_counts()

both          3568
left_only        0
right_only       0
Name: _merge, dtype: int64

In [148]:
df.isna().sum()

PDF URL                0
Summary                0
Title                  0
Author                 0
DOI                 3165
Published Date         0
Abstract               0
Journal Ref         3052
Primary Category       0
Category               0
Entry ID               0
_merge                 0
dtype: int64

In [149]:
df.shape

(3568, 12)

In [150]:
df['Summary'] = df['Summary'].map(lambda x: str(x).lower().replace('natural langauge processing', ''))

In [151]:
# Convert to list
data = df['Summary'].values.tolist()

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

In [152]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [153]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [154]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ']):#,  'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [155]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_trigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
#nlp = spacy.load('en', disable=['parser', 'ner'])
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

C:\Users\dglover\Anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['parser', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


In [156]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [157]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [158]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.018*"research" + 0.018*"domain" + 0.013*"technique" + 0.012*"deep" + '
  '0.010*"technology" + 0.010*"understand" + 0.010*"researcher" + '
  '0.010*"human" + 0.009*"develop" + 0.009*"document"'),
 (1,
  '0.015*"plm" + 0.014*"embedding" + 0.013*"improve" + 0.013*"accuracy" + '
  '0.012*"label" + 0.011*"prediction" + 0.011*"transfer" + 0.010*"arabic" + '
  '0.009*"multilingual" + 0.009*"pair"'),
 (2,
  '0.020*"large" + 0.020*"result" + 0.018*"training" + 0.017*"performance" + '
  '0.015*"train" + 0.014*"dataset" + 0.013*"pre_traine" + 0.012*"state" + '
  '0.011*"method" + 0.011*"work"'),
 (3,
  '0.021*"system" + 0.018*"propose" + 0.010*"speech" + 0.010*"process" + '
  '0.010*"method" + 0.009*"set" + 0.009*"generate" + 0.009*"generation" + '
  '0.009*"learn" + 0.009*"form"'),
 (4,
  '0.024*"semantic" + 0.023*"sentence" + 0.016*"network" + 0.014*"structure" + '
  '0.012*"graph" + 0.011*"word" + 0.010*"concept" + 0.009*"representation" + '
  '0.009*"compare" + 0.008*"query"')]


In [159]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.27452750945754

Coherence Score:  0.3509583166428295


In [160]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\dglover\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.168385  0.082582       1        1  31.073844
2      0.183784 -0.232332       2        1  24.941186
0      0.072375  0.221740       3        1  20.821549
4     -0.223403  0.077951       4        1  13.133244
1     -0.201141 -0.149941       5        1  10.030177, topic_info=                    Term        Freq       Total Category  logprob  loglift
246               system  958.000000  958.000000  Default  30.0000  30.0000
18                 large  778.000000  778.000000  Default  29.0000  29.0000
520             semantic  470.000000  470.000000  Default  28.0000  28.0000
29                result  784.000000  784.000000  Default  27.0000  27.0000
45                domain  563.000000  563.000000  Default  26.0000  26.0000
..                   ...         ...         ...      ...      ...      ...
153  machine_translation   81.517527   84.144964   Topic5  -5.2126   2.2678
107             resource  107.396221  125.803007   Topic5  -4.9369   2.1414
17               improve  192.951635  618.177964   Topic5  -4.3510   1.1352
815               sample   88.057711  138.984062   Topic5  -5.1354   1.8432
325           experiment   81.872119  169.137273   Topic5  -5.2082   1.5740

[260 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
2818      3  0.995097  academic
228       3  0.015557  accuracy
228       5  0.980096  accuracy
785       2  0.951539   achieve
785       5  0.046341   achieve
...     ...       ...       ...
584       4  0.561864      word
77        1  0.303961      work
77        2  0.580022      work
77        3  0.114536      work
374       3  0.995881     write

[339 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 1, 5, 2])

In [161]:
import pandas as pd

def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [162]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=texts)

C:\Users\dglover\AppData\Local\Temp\ipykernel_8012\1533496723.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)


In [163]:
df_topic_sents_keywords

,Dominant_Topic,Perc_Contribution,Topic_Keywords,0
0,3,0.6313,"system, propose, speech, process, method, set,...","[hadoop, platform, process, large, amount, req..."
1,3,0.3173,"system, propose, speech, process, method, set,...","[integrating, ai, plan, combination, explicit,..."
2,0,0.5041,"research, domain, technique, deep, technology,...","[tool, machine, rocesse, train, previously, an..."
3,2,0.5705,"large, result, training, performance, train, d...","[name, know, refer, ethnic, group, manipur, im..."
4,4,0.2966,"semantic, sentence, network, structure, graph,...","[note, course, introduce, reader, understand, ..."
...,...,...,...,...
3563,3,0.3075,"system, propose, speech, process, method, set,...","[automate, method, security, directive, presen..."
3564,3,0.4256,"system, propose, speech, process, method, set,...","[modifie, run, driven, augmentation, fully, la..."
3565,2,0.4774,"large, result, training, performance, train, d...","[large, llm, revolutionise, understanding, int..."
3566,0,0.3765,"research, domain, technique, deep, technology,...","[predictive, power, vary, clinical, note, type..."


In [164]:
df

,PDF URL,Summary,Title,Author,DOI,Published Date,Abstract,Journal Ref,Primary Category,Category,Entry ID,_merge
0,http://arxiv.org/pdf/1608.04434v1,hadoop is one of the platform s that can proce...,Natural Language Processing using Hadoop and K...,"[arxiv.Result.Author('Emre Erturk'), arxiv.Res...",NaN,2016-08-15 23:09:21+00:00,"Natural language processing, as a data analyti...",NaN,cs.CL,['cs.CL'],http://arxiv.org/abs/1608.04434v1,both
1,http://arxiv.org/pdf/2202.07138v2,integrating ai planning with natural language ...,Integrating AI Planning with Natural Language ...,"[arxiv.Result.Author('Kebing Jin'), arxiv.Resu...",NaN,2022-02-15 02:19:09+00:00,Natural language processing (NLP) aims at inve...,NaN,cs.AI,"['cs.AI', 'cs.CL']",http://arxiv.org/abs/2202.07138v2,both
2,http://arxiv.org/pdf/1906.11608v2,the tools are machine learning based using nat...,Simple Natural Language Processing Tools for D...,[arxiv.Result.Author('Leon Derczynski')],NaN,2019-06-27 13:15:12+00:00,This technical note describes a set of baselin...,NaN,cs.CL,['cs.CL'],http://arxiv.org/abs/1906.11608v2,both
3,http://arxiv.org/pdf/2006.16212v1,the name tangkhul also known as hao or ihao re...,Towards the Study of Morphological Processing ...,"[arxiv.Result.Author('Mirinso Shadang'), arxiv...",NaN,2020-06-29 17:24:09+00:00,There is no or little work on natural language...,In proceeding of Regional International Confer...,cs.CL,['cs.CL'],http://arxiv.org/abs/2006.16212v1,both
4,http://arxiv.org/pdf/1511.07916v1,this is a lecture note for the course ds ga at...,Natural Language Understanding with Distribute...,[arxiv.Result.Author('Kyunghyun Cho')],NaN,2015-11-24 23:23:13+00:00,This is a lecture note for the course DS-GA 30...,NaN,cs.CL,"['cs.CL', 'stat.ML']",http://arxiv.org/abs/1511.07916v1,both
...,...,...,...,...,...,...,...,...,...,...,...,...
3563,http://arxiv.org/pdf/2307.01211v1,an automated method for the ontologicalreprese...,An automated method for the ontological repres...,"[arxiv.Result.Author('Giampaolo Bella'), arxiv...",NaN,2023-06-30 09:04:47+00:00,Large documents written in juridical language ...,NaN,cs.AI,"['cs.AI', 'cs.CL']",http://arxiv.org/abs/2307.01211v1,both
3564,http://arxiv.org/pdf/2307.01488v1,scat modifies ran driven augmentations of the ...,SCAT: Robust Self-supervised Contrastive Learn...,"[arxiv.Result.Author('Junjie Wu'), arxiv.Resul...",NaN,2023-07-04 05:41:31+00:00,Despite their promising performance across var...,NaN,cs.CL,['cs.CL'],http://arxiv.org/abs/2307.01488v1,both
3565,http://arxiv.org/pdf/2307.06090v1,large language models llms have revolutionised...,Can Large Language Models Aid in Annotating Sp...,"[arxiv.Result.Author('Siddique Latif'), arxiv....",NaN,2023-07-12 11:27:40+00:00,Despite recent advancements in speech emotion ...,NaN,cs.SD,"['cs.SD', 'eess.AS']",http://arxiv.org/abs/2307.06090v1,both
3566,http://arxiv.org/pdf/2307.07051v1,predictive power varies with clinical note typ...,Making the Most Out of the Limited Context Len...,"[arxiv.Result.Author('Hongyi Zheng'), arxiv.Re...",NaN,2023-07-13 20:04:05+00:00,Recent advances in large language models have ...,Association for Computational Linguistics - St...,cs.CL,"['cs.CL', 'cs.IR', 'cs.LG']",http://arxiv.org/abs/2307.07051v1,both


In [165]:
# Assuming 'original_df' is your original dataframe and 'lda_output' is the output from LDA
merged_df = df.merge(df_topic_sents_keywords, left_index=True, right_index=True)

In [166]:
merged_df.head()

,PDF URL,Summary,Title,Author,DOI,Published Date,Abstract,Journal Ref,Primary Category,Category,Entry ID,_merge,Dominant_Topic,Perc_Contribution,Topic_Keywords,0
0,http://arxiv.org/pdf/1608.04434v1,hadoop is one of the platform s that can proce...,Natural Language Processing using Hadoop and K...,"[arxiv.Result.Author('Emre Erturk'), arxiv.Res...",NaN,2016-08-15 23:09:21+00:00,"Natural language processing, as a data analyti...",NaN,cs.CL,['cs.CL'],http://arxiv.org/abs/1608.04434v1,both,3,0.6313,"system, propose, speech, process, method, set,...","[hadoop, platform, process, large, amount, req..."
1,http://arxiv.org/pdf/2202.07138v2,integrating ai planning with natural language ...,Integrating AI Planning with Natural Language ...,"[arxiv.Result.Author('Kebing Jin'), arxiv.Resu...",NaN,2022-02-15 02:19:09+00:00,Natural language processing (NLP) aims at inve...,NaN,cs.AI,"['cs.AI', 'cs.CL']",http://arxiv.org/abs/2202.07138v2,both,3,0.3173,"system, propose, speech, process, method, set,...","[integrating, ai, plan, combination, explicit,..."
2,http://arxiv.org/pdf/1906.11608v2,the tools are machine learning based using nat...,Simple Natural Language Processing Tools for D...,[arxiv.Result.Author('Leon Derczynski')],NaN,2019-06-27 13:15:12+00:00,This technical note describes a set of baselin...,NaN,cs.CL,['cs.CL'],http://arxiv.org/abs/1906.11608v2,both,0,0.5041,"research, domain, technique, deep, technology,...","[tool, machine, rocesse, train, previously, an..."
3,http://arxiv.org/pdf/2006.16212v1,the name tangkhul also known as hao or ihao re...,Towards the Study of Morphological Processing ...,"[arxiv.Result.Author('Mirinso Shadang'), arxiv...",NaN,2020-06-29 17:24:09+00:00,There is no or little work on natural language...,In proceeding of Regional International Confer...,cs.CL,['cs.CL'],http://arxiv.org/abs/2006.16212v1,both,2,0.5705,"large, result, training, performance, train, d...","[name, know, refer, ethnic, group, manipur, im..."
4,http://arxiv.org/pdf/1511.07916v1,this is a lecture note for the course ds ga at...,Natural Language Understanding with Distribute...,[arxiv.Result.Author('Kyunghyun Cho')],NaN,2015-11-24 23:23:13+00:00,This is a lecture note for the course DS-GA 30...,NaN,cs.CL,"['cs.CL', 'stat.ML']",http://arxiv.org/abs/1511.07916v1,both,4,0.2966,"semantic, sentence, network, structure, graph,...","[note, course, introduce, reader, understand, ..."


In [167]:
merged_df.to_csv('../data/Dominant_topics_of_summaries_v2.csv')

# Archive

In [168]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=texts)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

In [ ]:
df_dominant_topic.to_csv('../data/Dominant_topics_of_absracts.csv')

In [ ]:
df_dominant_topic.tail(10)

In [ ]:
df_dominant_topic['Dominant_Topic'].value_counts()

In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

In [ ]:
# Display setting to show more characters in column
pd.options.display.max_colwidth = 100

sent_topics_sorteddf_mallet = pd.DataFrame()
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=False).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"]

# Show
sent_topics_sorteddf_mallet.head(10)

In [ ]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics